In [44]:
using BenchmarkTools
using StaticArrays
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [25]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [26]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "15445580690061347177", 2, "feature")

In [27]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [28]:
y = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "2330616016590698951", 2, "feature")

## Vecchia funzione

In [29]:
function box(model)
	if isa(model,Matrix)
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		#dim = checkStruct(listOfModels)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = box(listOfModels[1])
			for theModel in listOfModels[2:end]
				modelMin,modelMax = box(theModel)
				for (k,val) in enumerate(modelMin)
					if val < theMin[k]
						theMin[k]=val
					end
				end
				for (k,val) in enumerate(modelMax)
					if val > theMax[k]
						theMax[k]=val
					end
				end
			end
		end
		return [theMin,theMax]

	elseif (isa(model,Tuple) ||isa(model,Array))&& (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end

	return [theMin,theMax]
end

box (generic function with 1 method)

In [30]:
@btime box(x)

  6.580 μs (53 allocations: 2.72 KiB)


2-element Array{Array{Float64,2},1}:
 [0.0; 0.0]
 [1.0; 1.0]

In [31]:
@btime box(y)

  27.200 μs (118 allocations: 5.55 KiB)


2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]

In [32]:
@code_llvm box(y)


;  @ In[29]:1 within `box'
; Function Attrs: uwtable
define nonnull %jl_value_t* @japi1_box_2723(%jl_value_t*, %jl_value_t**, i32) #0 {
top:
  %3 = alloca %jl_value_t*, i32 3
  %gcframe = alloca %jl_value_t*, i32 12, align 16
  %4 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %4, i8 0, i32 96, i1 false)
  %5 = alloca %jl_value_t**, align 8
  store volatile %jl_value_t** %1, %jl_value_t*** %5, align 8
  %6 = alloca [1 x i8], align 1
  %.sroa.016 = alloca i8, align 1
  %7 = call %jl_value_t*** inttoptr (i64 1761836128 to %jl_value_t*** ()*)() #8
  %8 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 0
  %9 = bitcast %jl_value_t** %8 to i64*
  store i64 40, i64* %9
  %10 = getelementptr %jl_value_t**, %jl_value_t*** %7, i32 0
  %11 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 1
  %12 = bitcast %jl_value_t** %11 to %jl_value_t***
  %13 = load %jl_value_t**, %jl_value_t*** %10
  store %jl_value_t** %13, %jl_value_t*** %12
  %14 

In [33]:
@benchmark box(y)

BenchmarkTools.Trial: 
  memory estimate:  5.55 KiB
  allocs estimate:  118
  --------------
  minimum time:     26.699 μs (0.00% GC)
  median time:      49.800 μs (0.00% GC)
  mean time:        108.044 μs (0.72% GC)
  maximum time:     48.779 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Nuove funzioni

"""\
evalBox(listOfModels::Array)::Tuple\
Genera e restituisce  il volume di contenimento, ovvero il minimo parallelepipedo parallelo agli assi che li contiene.\
"""

In [34]:
function evalBox(listOfModels)
	theMin,theMax = box(listOfModels[1])
	for theModel in listOfModels[2:end]
		modelMin,modelMax= box(theModel)
		for (k,val) in enumerate(modelMin)
			if val < theMin[k]
				theMin[k]=val
			end
		end
		for (k,val) in enumerate(modelMax)
			if val > theMax[k]
				theMax[k]=val
			end
		end
	end
	return theMin,theMax
end

evalBox (generic function with 1 method)

"""\
box(model)::Array\
I box sono associati ai nodi, ovvero alle strutture. Sia il modello in ingresso una struttura, una tupla o un’array, restituisce [theMin,theMax] che rappresenta il volume di contenimento. Estrae quindi il solo volume di vista, caratterizzato dai parametri del box stesso(angolo di apertura, direzione asse, …).\
"""

In [35]:
function box(model)
	if isa(model,Matrix)
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		#dim = checkStruct(listOfModels)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = evalBox(listOfModels)
		end
        
		return [theMin,theMax]

	elseif (isa(model,Tuple) || isa(model,Array)) && (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end
    
	return [theMin,theMax]
end

box (generic function with 1 method)

In [36]:
@btime box(x)

  9.366 μs (53 allocations: 2.72 KiB)


2-element Array{Array{Float64,2},1}:
 [0.0; 0.0]
 [1.0; 1.0]

In [37]:
@btime box(y)

  23.900 μs (116 allocations: 5.48 KiB)


2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]

In [38]:
@benchmark box(y)

BenchmarkTools.Trial: 
  memory estimate:  5.48 KiB
  allocs estimate:  116
  --------------
  minimum time:     16.600 μs (0.00% GC)
  median time:      36.599 μs (0.00% GC)
  mean time:        45.275 μs (1.52% GC)
  maximum time:     6.976 ms (98.55% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [39]:
@code_typed box(y)

CodeInfo(
1 ──       goto #3 if not false
2 ──       nothing::Nothing
3 ┄─ %3  = Base.getproperty(Main.Lar, :Struct)::Any
│    %4  = (model isa %3)::Bool
└───       goto #12 if not %4
4 ── %6  = Base.getproperty(Main.Lar, :evalStruct)::Any
│    %7  = (%6)(model)::Any
│    %8  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Any,1}, svec(Any, Int64), 0, :(:ccall), Array{Any,1}, 0, 0))::Array{Any,1}
│    %9  = Main.:(==)::Core.Compiler.Const(==, false)
│    %10 = (isa)(%7, WeakRef)::Bool
└───       goto #6 if not %10
5 ── %12 = π (%7, WeakRef)
│    %13 = invoke %9(%12::WeakRef, %8::Array{Any,1})::Union{Missing, Bool}
└───       goto #9
6 ── %15 = (isa)(%7, Missing)::Bool
└───       goto #8 if not %15
7 ──       goto #9
8 ── %18 = (%7 == %8)::Union{Missing, Bool}
└───       goto #9
9 ┄─ %20 = φ (#5 => %13, #7 => $(QuoteNode(missing)), #8 => %18)::Union{Missing, Bool}
└───       goto #11 if not %20
10 ─ %22 = Base.getfield(model, :box)::Any
└───       return %22
11 ─ %24 = Main.evalBox(

## Modifiche

In [45]:
@inline function evalBox(listOfModels)
	theMin,theMax = box(listOfModels[1])
	for theModel in listOfModels[2:end]
		modelMin,modelMax= box(theModel)
		for (k,val) in enumerate(modelMin)
			if val < theMin[k]
				theMin[k]=val
			end
		end
		for (k,val) in enumerate(modelMax)
			if val > theMax[k]
				theMax[k]=val
			end
		end
	end
	return theMin,theMax
end

evalBox (generic function with 1 method)

In [46]:
@inline function box(model)
	if (isa(model,MMatrix) || isa(model,Matrix))
		return nothing
	elseif isa(model,Lar.Struct)
		listOfModels = Lar.evalStruct(model)
		if listOfModels == []
			return model.box
		else
			theMin,theMax = evalBox(listOfModels)
		end
        
		return [theMin,theMax]

	elseif (isa(model,Tuple) || isa(model,Array)) && (length(model)>=2)
		V = model[1]
		theMin = minimum(V, dims=2)
		theMax = maximum(V, dims=2)
	end
    
	return [theMin,theMax]
end

box (generic function with 1 method)

In [47]:
@btime box(x)

  6.160 μs (53 allocations: 2.72 KiB)


2-element Array{Array{Float64,2},1}:
 [0.0; 0.0]
 [1.0; 1.0]

In [48]:
@btime box(y)

  15.600 μs (116 allocations: 5.48 KiB)


2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]

In [49]:
@code_llvm box(y)


;  @ In[46]:1 within `box'
; Function Attrs: uwtable
define nonnull %jl_value_t* @japi1_box_3217(%jl_value_t*, %jl_value_t**, i32) #0 {
top:
  %3 = alloca %jl_value_t*, i32 2
  %gcframe = alloca %jl_value_t*, i32 4, align 16
  %4 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %4, i8 0, i32 32, i1 false)
  %5 = alloca %jl_value_t**, align 8
  store volatile %jl_value_t** %1, %jl_value_t*** %5, align 8
  %6 = alloca [1 x i8], align 1
  %.sroa.0 = alloca i8, align 1
  %7 = call %jl_value_t*** inttoptr (i64 1761836128 to %jl_value_t*** ()*)() #8
  %8 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 0
  %9 = bitcast %jl_value_t** %8 to i64*
  store i64 8, i64* %9
  %10 = getelementptr %jl_value_t**, %jl_value_t*** %7, i32 0
  %11 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 1
  %12 = bitcast %jl_value_t** %11 to %jl_value_t***
  %13 = load %jl_value_t**, %jl_value_t*** %10
  store %jl_value_t** %13, %jl_value_t*** %12
  %14 = bi

In [50]:
@benchmark box(y)

BenchmarkTools.Trial: 
  memory estimate:  5.48 KiB
  allocs estimate:  116
  --------------
  minimum time:     15.300 μs (0.00% GC)
  median time:      16.500 μs (0.00% GC)
  mean time:        23.430 μs (3.82% GC)
  maximum time:     9.042 ms (99.05% GC)
  --------------
  samples:          10000
  evals/sample:     1